In [120]:
import numpy as np
import wandb
import pandas as pd
import datetime

In [166]:
def get_h36m_mpjpe_results(
    artifact_name: str, record: str = "test/test_res", exp: str = "human36m"
):
    api = wandb.Api()
    artifact = api.artifact(artifact_name)

    runs = artifact.used_by()
    runs = filter(
        lambda x: x.state not in ["crashed", "killed", "failed"]
        and x.summary["_timestamp"] > datetime.datetime(2022, 10, 17).timestamp()
        and exp in x.name,
        runs,
    )

    mpjpes = np.array([run.summary[record] for run in runs])

    return mpjpes


def describe(data, title="cGNF", sigfig: dict = dict(mean=1, std=2)):
    print(title)
    df = pd.DataFrame(data)
    return (
        df.explode(list(df.columns))
        .astype("float")
        .describe()
        .loc[["mean", "std"]]
        .T.round(sigfig)
        .T
    )

In [178]:
def get_h36m_calibration_results(
    artifact_name: str, record: str = "test/test_res", exp: str = "calibration", idx=0
):
    api = wandb.Api()
    artifact = api.artifact(artifact_name)

    runs = artifact.used_by()
    runs = filter(
        lambda x: x.state not in ["crashed", "killed", "failed"]
        and x.summary["_timestamp"] > datetime.datetime(2022, 10, 17).timestamp()
        and exp in x.name,
        runs,
    )

    mpjpes = np.array([run.history()[record][idx] for run in runs])

    return mpjpes

In [144]:
mpjpes = {}

mpjpes["test"] = get_h36m_mpjpe_results(
    "ppierzc/propose_human36m/mpii-prod-no-mode:best"
)
mpjpes["hard"] = get_h36m_mpjpe_results(
    "ppierzc/propose_human36m/mpii-prod-no-mode:best", "hard/test_res"
)
mpjpes["occl"] = get_h36m_mpjpe_results(
    "ppierzc/propose_human36m/mpii-prod-no-mode:best", "occl/best_mpjpe"
)

In [145]:
describe(mpjpes)

cGNF


,test,hard,occl
mean,57.50,87.30,47.00
std,0.06,0.13,0.18


In [146]:
mpjpes = {}

mpjpes["test"] = get_h36m_mpjpe_results("ppierzc/propose_human36m/mpii-prod:best")
mpjpes["hard"] = get_h36m_mpjpe_results(
    "ppierzc/propose_human36m/mpii-prod:best", "hard/test_res"
)
mpjpes["occl"] = get_h36m_mpjpe_results(
    "ppierzc/propose_human36m/mpii-prod:best", "occl/best_mpjpe"
)

In [147]:
describe(mpjpes, title="cGNF w Lsample")

cGNF w Lsample


,test,hard,occl
mean,53.00,79.30,41.80
std,0.06,0.05,0.04


In [150]:
mpjpes = {}

mpjpes["test"] = get_h36m_mpjpe_results(
    "ppierzc/propose_human36m/mpii-prod-xlarge:best"
)
mpjpes["hard"] = get_h36m_mpjpe_results(
    "ppierzc/propose_human36m/mpii-prod-xlarge:best", "hard/test_res"
)
mpjpes["occl"] = get_h36m_mpjpe_results(
    "ppierzc/propose_human36m/mpii-prod-xlarge:best", "occl/best_mpjpe"
)

In [151]:
describe(mpjpes, title="cGNF xlarge w Lsample")

cGNF xlarge w Lsample


,test,hard,occl
mean,48.50,72.60,39.90
std,0.02,0.09,0.05


In [179]:
cal = {}

cal["test"] = get_h36m_calibration_results(
    "ppierzc/propose_human36m/mpii-prod-xlarge:best",
    record="calibration_score",
    exp="calibration",
)
cal["occl"] = get_h36m_calibration_results(
    "ppierzc/propose_human36m/mpii-prod-xlarge:best",
    record="calibration_score",
    exp="calibration",
    idx=1,
)

In [182]:
describe(cal, title="cGNF xlarge w Lsample", sigfig=dict(mean=2, std=3))

cGNF xlarge w Lsample


,test,occl
mean,0.23,0.070
std,0.00,0.001


In [187]:
cal = {}

cal["test"] = get_h36m_calibration_results(
    "ppierzc/propose_human36m/mpii-prod:best",
    record="calibration_score",
    exp="calibration",
)
cal["occl"] = get_h36m_calibration_results(
    "ppierzc/propose_human36m/mpii-prod:best",
    record="calibration_score",
    exp="calibration",
    idx=1,
)

In [188]:
describe(cal, title="cGNF w Lsample", sigfig=dict(mean=2, std=3))

cGNF w Lsample


,test,occl
mean,0.08,0.03
std,0.00,0.00


In [192]:
cal = {}

cal["test"] = get_h36m_calibration_results(
    "ppierzc/propose_human36m/mpii-prod-no-mode:best",
    record="calibration_score",
    exp="calibration",
)
cal["occl"] = get_h36m_calibration_results(
    "ppierzc/propose_human36m/mpii-prod-no-mode:best",
    record="calibration_score",
    exp="calibration",
    idx=1,
)

In [193]:
describe(cal, title="cGNF", sigfig=dict(mean=2, std=3))

cGNF


,test,occl
mean,0.08,0.070
std,0.00,0.001
